Imports:

In [2]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


# Data Retrieval 

In [9]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

print("Example Dialogue:")
dialogue.sample(4)


Example Dialogue:


,text,speaker,season,episode,scene,utterance
19978,"Well, I'm so sorry. I thought I was leaving yo...",Phoebe Abbott,4,1,2,12
12076,"Joey, a woman just stuck her tounge down my th...",Chandler Bing,2,22,5,6


In [10]:
episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

print("Example Episode:")
episodes.sample(4)

Example Episode:


,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
164,7,19,The One with Ross and Monica's Cousin,Gary Halvorson,Andrew Reich & Ted Cohen,2001-04-19,16.55,8.7
182,8,13,The One Where Chandler Takes a Bath,Ben Weiss,Vanessa McCarthy,2002-01-17,29.24,8.6


# Inference Task

In [1]:
# I want to verify that lots of exclamation changes episode IMDb rating 

# Resampling import to use in bootstrapping 
from sklearn.utils import resample

# Two populations: IMDb ratings and Viewership 
df_regr = (master_list
    .groupby(['episode', 'season'])
    .agg({'Speaker_Exclaim': 'sum', 'Speaker_Exclaimed_Toward': 'sum', 'us_views_millions': 'mean', 'imdb_rating': 'mean'})
    .reset_index()
)
mean_exclamation = np.mean(df['Speaker_Exclaim'])
pop1 = df[df['Speaker_Exclaim']>mean_exclamation]['imdb_rating']
pop2 = df['imdb_rating']

# Define number of bootstrap resamples
n_resamples = 1000
mean_diffs = np.zeros(n_resamples)

#  Bootstrapping procedure (add difference in each resample mean to the list)
for i in range(n_resamples):
    s1 = resample(pop1, replace=True)
    s2 = resample(pop2, replace=True)
    mean_diffs[i] = np.mean(s1)-np.mean(s2) 
    
# Confidence interval 
ci_95 = np.percentile(mean_diffs, [2.5, 97.5]) 

# Mean Difference 
mean_difference = np.mean(mean_diffs)

